In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table
from dash.dependencies import Input, Output
import base64


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from AnimalShelter import AnimalShelter



###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "mypassword"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object and accept projection json input
df = pd.DataFrame.from_records(shelter.read({}))
df = df.drop_duplicates()
df.columns = ['#',
              'Age Upon Outcome',
              'Animal Id',
              'Animal Type',
              'Breed',
              'Color',
              'Date of Birth',
              'Date/Time',
              'Month/Year',
              'Name',
              'Outcome-Subtype',
              'Outcome-Type',
              'Sex Upon Outcome',
              'Location: Latitude',
              'Location: Longitude',
              'Age Upon Outcome in Weeks' ]



#########################
# Dashboard Layout / View
#########################

app = JupyterDash('SNHU CS-340 Dashboard')

#Update filepath to get image to load
image_filename = '/home/joshua.ryther_snhu/Desktop/ProjectTwo/assets/Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    #Header
    html.Div(id='hidden-div', style={'display':'none'}),
    html.A(html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
                         style={'height': '10%', 'width': '10%'})),href='https://www.snhu.edu'),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Center(html.B(html.P('Josh Ryther'))),
    html.Hr(),
    #Dropdown
    html.Div([
        html.P("Rescue Type:"),
        dcc.Dropdown(
            id='dropdown',
            options=[
                {'label': 'All', 'value': 'all'},
                {'label': 'Water', 'value': 'water'},
                {'label': 'Mountain', 'value': 'mountain'},
                {'label': 'Disaster', 'value': 'disaster'}
            ],
            clearable=False,
            value = 'all',
        )],
        style={"width": "20%"},
    ),
    #Data table
    dash_table.DataTable(
        id='datatable-id',
        style_cell={
        'whiteSpace': 'normal',
        'height': 'auto',
        'minWidth': '180px', 'width': '180px', 'maxWidth': '180px',
        'textAlign': 'left',
        },
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": False} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        sort_by=[{'column_id': '#', 'direction': 'asc'}],
        column_selectable="single",
        row_selectable='single',
        row_deletable=False,
        selected_columns=[],
        selected_rows=[1],
        page_action="native",
        page_current= 0,
        page_size= 10,
    ),
    html.Br(),
     html.Hr(),
    #Map
    
    html.Div(children = [
        html.Div(dcc.Graph(id="pie-chart"), style={'height': '500px', 'width': '49%','display': 'inline-block'}),
        html.Div(id='map-id', style={'height': '500px', 'width': '49%','display': 'inline-block'}),
    ], style={'height': '500px', 'width': '100%','display': 'inline-block'}),
])

#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
     Input('datatable-id', 'selected_rows')])

def update_map(viewData, selected_rows):
    dff = pd.DataFrame.from_dict(viewData)

    index = selected_rows[0]
    coord_lat = dff.iloc[index, 13]
    coord_long = dff.iloc[index, 14]
    
    return [
        dl.Map(style={'height': '500px','width': '100%'}, center=[coord_lat,coord_long], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[coord_lat,coord_long], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[index,9])
                ])
            ])
        ])
    ]

@app.callback([Output('datatable-id', "data"),
              Output("pie-chart", "figure")],
              [Input('dropdown', 'value')])

def update_table_chart(value):
    dff = df
    
    if (value == 'all'):
        data=dff.to_dict('records')
        fig = px.pie(
            data_frame = dff,
            names = "Animal Type",
            hole = .3,
            )
        
    elif (value == 'water'):
        dff = dff[dff["Breed"].isin(['Labrador Retriever Mix', 'Chesa Bay Retr Mix', 'Newfoundland Mix'])]
        dff = dff[dff["Sex Upon Outcome"].isin(['Intact Female'])]
        dff = dff[(dff["Age Upon Outcome in Weeks"] >= 26) & (dff["Age Upon Outcome in Weeks"] <= 156)]
        data=dff.to_dict('records')
        fig = px.pie(
            data_frame = dff,
            names = "Breed",
            hole = .3,
            )
        
    elif (value == 'mountain'):
        dff = dff[dff["Breed"].isin(['German Shepard', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler'])]
        dff = dff[dff["Sex Upon Outcome"].isin(['Intact Male'])]
        dff = dff[(dff["Age Upon Outcome in Weeks"] >= 26) & (dff["Age Upon Outcome in Weeks"] <= 156)]
        data=dff.to_dict('records')
        fig = px.pie(
            data_frame = dff,
            names = "Breed",
            hole = .3,
            )

    elif (value == 'disaster'):
        dff = dff[dff["Breed"].isin(['Doberman Pinsch', 'German Shepard', 'Golden Retriever', 'Bloodhound', 'Rottweiler'])]
        dff = dff[dff["Sex Upon Outcome"].isin(['Intact Male'])]
        dff = dff[(dff["Age Upon Outcome in Weeks"] >= 20) & (dff["Age Upon Outcome in Weeks"] <= 300)]
        data=dff.to_dict('records')
        fig = px.pie(
            data_frame = dff,
            names = "Breed",
            hole = .3,
            )
   
    return data, fig

app
